In [253]:
!pip install gdown tabulate
!pip install typing-extensions --upgrade
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=b85c10c20f0f305e656cdafb312f6bd8a6e88077ab3e9fc4add18845c3fea364
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [254]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import copy
import gdown
import os
from tabulate import tabulate
import tensorflow as tf
# import tf_agents
# from tf_agents.environments import suite_gym
# from tf_agents.environments import tf_py_environment
# from tf_agents.agents.dqn import dqn_agent
# from tf_agents.networks.q_network import QNetwork
# from tf_agents.replay_buffers import tf_uniform_replay_buffer
# from tf_agents.utils import common
# from tf_agents.drivers import dynamic_step_driver
# from tf_agents.policies import random_tf_policy
# from tf_agents.trajectories import trajectory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split
import pickle



In [255]:
from google.colab import drive
drive.mount('/content/drive')


# Define file path
file_path = "data_hackaton_train.data"

# Check if file exists before downloading
if not os.path.exists(file_path):
    print(f"Downloading {file_path}...")
    gdown.download('https://drive.google.com/uc?id=18HCzpRnUzMYwrWJ4k6qnkgqhlLY735NO', file_path, quiet=False)
else:
    print(f"{file_path} already exists. Skipping download.")
train_df = pd.read_pickle('/content/data_hackaton_train.data')

url = 'https://github.com/devpatel30/amr/raw/main/test_data.pickle'

# Download the file
!wget {url} -O test_data.pickle

global test_data
with open('test_data.pickle', 'rb') as f:
    test_dat = pd.read_pickle(f)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data_hackaton_train.data already exists. Skipping download.
--2025-03-16 17:24:43--  https://github.com/devpatel30/amr/raw/main/test_data.pickle
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/devpatel30/amr/main/test_data.pickle [following]
--2025-03-16 17:24:44--  https://raw.githubusercontent.com/devpatel30/amr/main/test_data.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24524011 (23M) [application/octet-stream]
Saving to: ‘test_data.pickle’

test_data.pi

In [ ]:
TARGET_LENGTH = 20

# 1) Find the maximum number of sequences across all rows
max_num_seqs = max(len(row['start']) for _, row in train_df.iterrows())
max_num_seqs = max(max_num_seqs, max(len(row['solution']) for _, row in train_df.iterrows()))

for i, row in train_df.iterrows():
    # --- Fix the number of sequences in 'start' ---
    if len(row['start']) < max_num_seqs:
        # pad with sequences of all dashes
        needed = max_num_seqs - len(row['start'])
        row['start'].extend(['-' * TARGET_LENGTH] * needed)
    else:
        # if too many sequences, truncate
        row['start'] = row['start'][:max_num_seqs]

    # --- Fix the number of sequences in 'solution' ---
    if len(row['solution']) < max_num_seqs:
        needed = max_num_seqs - len(row['solution'])
        row['solution'].extend(['-' * TARGET_LENGTH] * needed)
    else:
        row['solution'] = row['solution'][:max_num_seqs]

    # --- Now fix each sequence to length 20 ---
    for j, seq in enumerate(row['start']):
        if len(seq) < TARGET_LENGTH:
            row['start'][j] = seq.ljust(TARGET_LENGTH, '-')
        elif len(seq) > TARGET_LENGTH:
            row['start'][j] = seq[:TARGET_LENGTH]

    for j, seq in enumerate(row['solution']):
        if len(seq) < TARGET_LENGTH:
            row['solution'][j] = seq.ljust(TARGET_LENGTH, '-')
        elif len(seq) > TARGET_LENGTH:
            row['solution'][j] = seq[:TARGET_LENGTH]


# save first 100 lines as a csv
train_df.head(100).to_csv('train_df.csv', index=False)
train_df.shape

In [ ]:
train_df

# Get row 4 (using 0-based indexing, row 4 is index 4)
row = train_df.iloc[4]

# Print the number of sequences in start and solution
print("Row 4 - Number of sequences in 'start':", len(row['start']))
print("Row 4 - Number of sequences in 'solution':", len(row['solution']))

# Optionally, print each sequence length for a more detailed look
print("\nLengths of each sequence in 'start':")
for idx, seq in enumerate(row['start']):
    print(f"Sequence {idx}: length = {len(seq)}")

print("\nLengths of each sequence in 'solution':")
for idx, seq in enumerate(row['solution']):
    print(f"Sequence {idx}: length = {len(seq)}")




In [ ]:
DICT_SYMBOLS = {
    'A': 1,
    'T': 2,
    'C': 3,
    'G': 4}

In [ ]:
class Puzzle:
    def __init__(self, data):
        self.start = list(data["start"])
        self.moves = copy.deepcopy(data.get("moves"))
        self.steps = copy.deepcopy(data.get("steps"))
        self.solution = list(data["solution"])
        self.score = data["score"]
        self.accepted_pair = data["accepted_pair"]

        self.padded_start = self.build_puzzle_to_end(self.start)
        self.padded_solution = self.build_puzzle_to_end(self.solution)

    def build_puzzle_to_end(self, puzzle):
        """Pad each row with '-' to match the longest row for visualization."""
        max_len = max(len(row) for row in puzzle)
        return [row.ljust(max_len, '-') for row in puzzle]

    def plot_puzzle(self, puzzle, title):
        puzzle = self.build_puzzle_to_end(puzzle)
        num_puzzle = np.array([[DICT_SYMBOLS.get(char, 0) for char in row] for row in puzzle])
        rot_num_puzzle = np.rot90(num_puzzle, 1)
        puzzle_array = np.array([list(row) for row in puzzle])
        rot_labels = np.rot90(puzzle_array, 1)

        plt.figure(figsize=(8, 6))
        sns.heatmap(rot_num_puzzle, annot=rot_labels, fmt="", cmap="Pastel1_r", cbar=False)
        plt.title(title)
        plt.axis("off")
        plt.show()

    def gearbox_score(self, puzzle, bonus=1.15):
        consensus = self.accepted_pair
        score = 0
        for col_ind in range(len(puzzle[0])):
            col_bonus = True
            col_tot = 0
            column_chars = [row[col_ind] for row in puzzle]
            for char in column_chars:
                if char == "-":
                    col_bonus = False
                    continue
                if char in consensus[col_ind]:
                    col_tot += 1
                else:
                    col_bonus = False
            column_score = col_tot * bonus if col_bonus else col_tot
            score += column_score
        print(f"Total Gearbox Score: {score}")
        return score

    def _apply_step_to_puzzle(self, puzzle, step):
                """Apply a single step to the puzzle."""
                new_puzzle = puzzle.copy()
                # Convert step elements to integers if they are strings
                # Check if step is in the expected format before converting to int
                if isinstance(step, list) and len(step) == 2 and step[0].isdigit() and step[1].isdigit():
                    row_index = int(step[0]) - 1
                    col_index = int(step[1])
                else:
                    # Handle invalid steps, e.g., skip them or raise a more informative error
                    # print(f"Warning: Invalid step format: {step}. Skipping this step.")
                    return new_puzzle

                if row_index < 0 or row_index >= len(new_puzzle):
                    return new_puzzle
                row_str = new_puzzle[row_index]
                if col_index < 0 or col_index > len(row_str):
                    return new_puzzle
                new_row = row_str[:col_index] + '-' + row_str[col_index:]
                new_row = new_row[:len(row_str)]
                new_puzzle[row_index] = new_row
                return new_puzzle

    def apply_all_steps(self):
        """Apply all steps on a copy of the puzzle and plot states."""
        current_puzzle = list(self.start)
        updated_puzzles = []
        scores = []

        for step in self.steps:
            current_puzzle = self._apply_step_to_puzzle(current_puzzle, step)
            padded_current = self.build_puzzle_to_end(current_puzzle)
            score = self.gearbox_score(padded_current)
            updated_puzzles.append(padded_current)
            scores.append(score)

        n_steps = len(updated_puzzles)
        fig, axes = plt.subplots(1, n_steps, figsize=(4 * n_steps, 6))
        if n_steps == 1:
            axes = [axes]
        for idx, (puzzle_state, score) in enumerate(zip(updated_puzzles, scores)):
            num_puzzle = np.array([[DICT_SYMBOLS.get(char, 0) for char in row] for row in puzzle_state])
            rot_num_puzzle = np.rot90(num_puzzle, 1)
            puzzle_array = np.array([list(row) for row in puzzle_state])
            rot_labels = np.rot90(puzzle_array, 1)
            ax = axes[idx]
            sns.heatmap(rot_num_puzzle, annot=rot_labels, fmt="", cmap="Pastel1_r", cbar=False, ax=ax)
            ax.set_title(f"Step {idx+1}\nScore: {score}")
            ax.axis("off")
        plt.tight_layout()
        plt.show()
        return current_puzzle

In [ ]:
# Create puzzle
row = 3

#print the row
print(tabulate(train_df.iloc[[row]], headers='keys', tablefmt='pretty'))


puzzle_data = {
    'start': train_df.iloc[row]['start'],
    'moves': train_df.iloc[row].get('moves'),
    'steps': train_df.iloc[row].get('steps'),
    'solution': train_df.iloc[row]['solution'],
    'score': train_df.iloc[row]['score'],
    'accepted_pair': train_df.iloc[row]['accepted_pair']
}

puzzle = Puzzle(puzzle_data)

In [ ]:
puzzle.plot_puzzle(puzzle.start, "Starting Puzzle")
puzzle.plot_puzzle(puzzle.solution, "Solution Puzzle")

In [ ]:
updated_puzzle = puzzle.apply_all_steps()

In [ ]:
puzzle.steps

# Attempt at ML

In [ ]:
import gym
from gym import spaces
import numpy as np
import ast
import json
import pandas as pd
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim

class MSADataEnv(gym.Env):
    def __init__(self, data):
        super(MSADataEnv, self).__init__()
        self.data = data  # use the passed in DataFrame (train_data or val_data)
        self.current_index = 0

        # Compute the max sequence length from the dataset
        all_lengths = []
        for idx, row in self.data.iterrows():
            for seq_str in row['start']:  # row['start'] is already a list
                all_lengths.append(len(seq_str))
        self.max_length = max(all_lengths)  # Store max sequence length

        self.num_sequences = len(self.data.iloc[0]['start'])  # Count number of sequences in a row

        self.gap_insertions = np.zeros((self.num_sequences, self.max_length), dtype=int) # to count the gaps


        # Define observation space with fixed max_length
        self.observation_space = spaces.Box(
            low=-1, high=3,
            shape=(self.num_sequences, self.max_length),
            dtype=np.int32
        )

        # Define the action space: num_sequences * (max_length+1) possible gap insertions
        self.total_actions = self.num_sequences * (self.max_length + 1)
        self.action_space = spaces.Discrete(self.total_actions)
        self.MAX_STEPS = 10  # or another appropriate number
        self.current_step = 0

    def reset(self):
        self.current_step = 0  # Reset step counter at the start of an episode
        self.current_index = random.randint(0, len(self.data) - 1)
        row = self.data.iloc[self.current_index]
        self.state = self._process_start(row['start'])
        self.target_solution = self._process_solution(row['solution'])
        self.accepted_pair = row['accepted_pair']  # Add this line
        return self.state.copy()


    def step(self, action):
        seq_idx = action // (self.max_length + 1)
        pos = action % (self.max_length + 1)

        sequence = list(self.state[seq_idx])
        # Insert gap
        sequence.insert(pos, -1)
        if len(sequence) > self.max_length:
            sequence = sequence[:self.max_length]
        self.state[seq_idx] = np.array(sequence)

        # *DEBUG PRINT* to see if state is changing
        # print(f"After action {action}: seq {seq_idx}, pos {pos}")
        # for i, row in enumerate(self.state):
        #     print(f" Seq {i}: {row}")

        self.current_step += 1
        done = self.current_step >= self.MAX_STEPS
        reward = self._calculate_reward(self.state, self.target_solution)

        return self.state.copy(), reward, done, {}



    def _process_start(self, start_data):
        mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3, '-': -1}
        sequences = []
        for seq_str in start_data:
            numeric_seq = [mapping[ch] for ch in seq_str]
            # Pad to length 12
            while len(numeric_seq) < self.max_length:
                numeric_seq.append(-1)
            # Truncate if longer than 12
            numeric_seq = numeric_seq[:self.max_length]
            sequences.append(numeric_seq)

        # If start_data has fewer than self.num_sequences, pad the extra sequences
        # If more, truncate. The end result: shape = (self.num_sequences, self.max_length)
        if len(sequences) < self.num_sequences:
            for _ in range(self.num_sequences - len(sequences)):
                sequences.append([-1]*self.max_length)
        sequences = sequences[:self.num_sequences]

        return np.array(sequences)




    def _process_solution(self, solution_data):
      mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3, '-': -1}
      sequences = []
      for seq_str in solution_data:
          numeric_seq = [mapping[ch] for ch in seq_str]

          # Pad up to self.seq_length (e.g., 12)
          while len(numeric_seq) < self.max_length:
              numeric_seq.append(-1)

          # Truncate if it exceeds self.seq_length
          numeric_seq = numeric_seq[:self.max_length]

          sequences.append(numeric_seq)

      return np.array(sequences)

    def _calculate_reward(self, state, target):
        # Base reward: fraction of matching positions (range: 0 to 1)
        return np.mean(state == target)





In [ ]:
# Define the Q-Network (a simple feed-forward network)
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )

    def forward(self, x):
        return self.net(x)



In [ ]:
# Hyperparameters
# Hyperparameters
BATCH_SIZE      = 128
GAMMA           = 0.99
EPS_START       = 0.8
EPS_END         = 0.2
EPS_DECAY       = 0.999   # approximate exponential decay
TARGET_UPDATE   = 128
LR              = 1e-4
MEMORY_SIZE     = 100000
NUM_EPISODES    = 250   # 3250 seems to be best

In [ ]:
from sklearn.model_selection import train_test_split

# Split your training DataFrame into 80% train and 20% validation
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)
#download some data from github and read it



# Replay Memory
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# Prepare environment and networks
env = MSADataEnv(train_data)
state_dim = env.num_sequences * env.max_length  # Flatten state
action_dim = env.total_actions

policy_net = DQN(state_dim, action_dim)
target_net = DQN(state_dim, action_dim)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=LR)
memory = ReplayMemory(MEMORY_SIZE)

# Epsilon for epsilon-greedy policy
epsilon = EPS_START

# Function to select an action
def select_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state.flatten()).unsqueeze(0)
            q_values = policy_net(state_tensor)
            return q_values.max(1)[1].item()



**Training**

In [ ]:
reward_history = []


# Training loop
for episode in range(NUM_EPISODES):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward

        memory.push(state, action, reward, next_state, done)
        state = next_state

        # If we have enough samples, perform optimization
        if len(memory) >= BATCH_SIZE:
            transitions = memory.sample(BATCH_SIZE)
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = zip(*transitions)

            batch_state = torch.FloatTensor([s.flatten() for s in batch_state])
            batch_action = torch.LongTensor(batch_action).unsqueeze(1)
            batch_reward = torch.FloatTensor(batch_reward).unsqueeze(1)
            batch_next_state = torch.FloatTensor([s.flatten() for s in batch_next_state])
            batch_done = torch.FloatTensor(batch_done).unsqueeze(1)

            # Compute current Q values
            current_q = policy_net(batch_state).gather(1, batch_action)
            # Compute next Q values from target network
            next_q = target_net(batch_next_state).max(1)[0].detach().unsqueeze(1)
            # Compute expected Q values
            expected_q = batch_reward + (GAMMA * next_q * (1 - batch_done))

            loss = nn.MSELoss()(current_q, expected_q)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()



    # Decay epsilon
    epsilon = max(epsilon * EPS_DECAY, EPS_END)

    # Update target network
    if episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

    print(f"Episode {episode+1}/{NUM_EPISODES}, Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.4f}")
    reward_history.append(total_reward)



print("Training complete.")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


plt.plot(reward_history)  # reward_history = list of total rewards per episode
plt.xlabel("Episodes")
plt.ylabel("Total Reward")
plt.title("Training Progress")
plt.show()

smoothed = pd.Series(reward_history).rolling(100).mean()
smoothed.plot()



In [ ]:
def evaluate_agent(env, agent, num_episodes=100):
    """
    Evaluate the agent for a specified number of episodes.
    Returns the average accuracy and a list of steps for each episode.
    """
    total_accuracy = 0.0
    agent.eval()  # Put the agent in evaluation mode
    all_episodes_steps = []  # We'll store steps for each episode here

    with torch.no_grad():
        for episode_idx in range(num_episodes):
            # Reset environment
            state = env.reset()
            done = False
            episode_steps = []  # This will collect (seq_idx, pos) for the current episode

            while not done:
                # Let the agent select an action (no exploration in eval)
                action = select_action(state, epsilon=0.05)

                # Decode the action into (seq_idx, pos) for logging
                seq_idx = action // (env.max_length + 1)
                pos = action % (env.max_length + 1)
                episode_steps.append((seq_idx, pos))

                # Step the environment
                state, reward, done, _ = env.step(action)

            # After the episode finishes, accumulate the reward
            total_accuracy += reward

            # Store steps for this episode
            all_episodes_steps.append(episode_steps)

    # Compute average accuracy
    avg_accuracy = total_accuracy / num_episodes

    return avg_accuracy, all_episodes_steps


# Then, call it like so:
env_val = MSADataEnv(test_dat)
accuracy, episodes_steps = evaluate_agent(env_val, policy_net, num_episodes=10)

print(f"Average Accuracy: {accuracy*100:.2f}%")

def remove_duplicates(steps):
    seen = set()
    unique_steps = []
    for step in steps:
        if step not in seen:
            unique_steps.append(step)
            seen.add(step)
    return unique_steps

unique_episodes_steps = [remove_duplicates(steps) for steps in episodes_steps]

# Print out the steps for each episode
for i, step_list in enumerate(unique_episodes_steps):
    print(f"Episode {i}: {step_list}")



In [ ]:
# 1. Extract the row you want
row_4 = train_df.iloc[[1]]  # Notice the double brackets so it's still a DataFrame

# 2. Create an environment with just this row
env_test_row4 = MSADataEnv(row_4)

# 3. Evaluate your agent on that single row
accuracy_row4, steps_row4 = evaluate_agent(env_test_row4, policy_net, num_episodes=1)

print(f"Accuracy for row 4: {accuracy_row4 * 100:.2f}%")
print("Steps taken (row 4):", steps_row4[0])  # steps_row4 is a list of lists; we only have 1 episode


In [ ]:
def gearbox_score(accepted_pair, puzzle, bonus=1.15):
    """
    Calculate and print the total gearbox score based on a puzzle grid and accepted character pairs.

    The function iterates through each column of the puzzle, summing up occurrences of characters that
    are present in the consensus (accepted_pair) for that specific column. For each column, if every
    non-dash character matches the accepted consensus, a bonus multiplier is applied to the column score.

    Parameters:
        accepted_pair (list of iterables): Each element contains the accepted characters for the
                                             corresponding column in the puzzle.
        puzzle (list of list of str): A 2D grid representing the puzzle, where each inner list is a row.
        bonus (float, optional): The bonus multiplier applied to a column's score if all characters (except dashes)
                                 match the accepted consensus. Defaults to 1.15.

    Returns:
        float: The total gearbox score calculated by summing the column scores, with bonuses applied as appropriate.
    """
    consensus = accepted_pair
    score = 0
    for col_ind in range(len(puzzle[0])):
        col_bonus = True
        col_tot = 0
        column_chars = [row[col_ind] for row in puzzle]
        for char in column_chars:
            if char == "-":
                col_bonus = False
                continue
            if char in consensus[col_ind]:
                col_tot += 1
            else:
                col_bonus = False
        column_score = col_tot * bonus if col_bonus else col_tot
        score += column_score
    print(f"Total Gearbox Score: {score}")
    return score

#test with row 4
gearbox_score(row_4['accepted_pair'].values[0], row_4['start'].values[0])


Heat Map for the OG data

In [ ]:
from collections import Counter

# Extract gap insertions from "steps" column
# Example format of steps: [(0,3), (1,5), (0,3), (2,1)]
gap_positions = []
for step_list in train_df['steps']:
    # Check if step_list is None before iterating
    if step_list is not None:  # Skip if step_list is None
        for seq_idx, pos in step_list:
            gap_positions.append((seq_idx, pos))

# Count most common gap insertions
gap_freq = Counter(gap_positions)

# Convert to DataFrame for visualization
gap_df = pd.DataFrame(gap_freq.items(), columns=['Position', 'Frequency'])
gap_df[['Sequence Index', 'Insertion Position']] = pd.DataFrame(gap_df['Position'].tolist(), index=gap_df.index)
gap_df.drop(columns=['Position'], inplace=True)

# Plot Heatmap of Frequent Gap Insertions
heatmap_data = np.zeros((max(gap_df['Sequence Index'])+1, max(gap_df['Insertion Position'])+1))

for _, row in gap_df.iterrows():
    heatmap_data[row['Sequence Index'], row['Insertion Position']] = row['Frequency']

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap='Blues', annot=True, fmt=".0f")
plt.xlabel("Insertion Position")
plt.ylabel("Sequence Index")
plt.title("Gap Insertion Frequency Heatmap")
plt.show()

Trying heatmap from the ML

Test

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_gap_insertions(env):
#     plt.figure(figsize=(10, 6))
#     sns.heatmap(env.gap_insertions, cmap="Blues", annot=True, fmt=".0f", cbar=True)

#     plt.xlabel("Position in Sequence")
#     plt.ylabel("Sequence Index")
#     plt.title("Frequency of Gap Insertions by AI")
#     plt.show()

# # After running several episodes
# plot_gap_insertions(env)